# Homework 2: Random Walks and Related Creatures

First, imports:

In [32]:
import random
import math

1. A gambler with 100 is placing bets. On each bet, the gambler wins 1 with
probability p or loses 1 with probability q = 1 − p. If the gambler reaches 200,
they cash out, and if they reach 0, they go broke; in either of these cases they
stop playing.
(a) If p = 0.5, what is the probability that the gambler goes broke? When the
gambler goes broke, on average how long does that take?

In [33]:
def bet(p):
    # Winning condition
    if random.random() <= p:
        return True
    # Otherwise, you lose a dollar
    else:
        return False

def experiment(M, p):
    bets = 0
    # While the gambler has less money than $200 and is not broke
    while 0 < M < 200:
        # If the gambler wins the bet, add a dollar
        if bet(p):
            M += 1
        # If the gambler loses the bet, subtract a dollar
        else:
            M -= 1
        # Count the number of rounds
        bets += 1
    # If gambler goes broke
    if M == 0:
        return bets, True
    # If the gambler gets to $200
    else:
        return bets, False

money = 100
prob = 0.5
N = 100000
data = [experiment(money, prob) for i in range(N)]
# For calculating the probability that the gambler goes broke
broke = []
# Number of rounds until the gambler went broke
roundsToBroke = []
for i in data:
    # Append all times the gambler either went broke or got to $200 to this array
    broke.append(i[1])
    # If the gambler went broke, append the number of rounds it took to get to broke
    if i[1]:
        roundsToBroke.append(i[0])
print("Probability of going broke with p = 0.5: ", sum(broke)/N)
print("Average number of rounds before going broke: ", sum(roundsToBroke)/N)

Probability of going broke with p = 0.5:  0.50154
Average number of rounds before going broke:  4994.97058


(b) If p = 0.3, same question.

In [34]:
money = 100
prob = 0.3
N = 100000
data = [experiment(money, prob) for i in range(N)]
broke = []
roundsToBroke = []
for i in data:
    broke.append(i[1])
    if i[1]:
        roundsToBroke.append(i[0])
print("Probability of going broke with p = 0.3: ", sum(broke)/N)
print("Average number of rounds before going broke: ", sum(roundsToBroke)/N)

Probability of going broke with p = 0.3:  1.0
Average number of rounds before going broke:  250.09482


2. Prices of certain financial instruments are modeled by a type of random walk called
Brownian motion. These models date back to Bachelier’s now-famous paper in
1900, Theorie de la speculation, which detailed the first use of stochastic models
in modeling the price of stock options. In this problem, you’ll take a look at a
simple stock option, but we’ll describe it as a bet.
There is a changing number S (score? stock?) we are tracking, and you are
betting on what this number will be in thirty days. The number increases by 1
with probability p or decreases by 1 with probability q = 1 − p each day. You may
place a bet right now on what this number will be at the end of 30 days; you have
to pay B to place the bet. If the number S is greater than 110, you win S − 110.
Otherwise, you lose the amount B that you bet.
(a) If p = 0.5 and B = 5, is this a good bet? Explain.

In [35]:
# function to decide if s goes up or down
def dailyMovement(p):
    if random.random() <= p:
        return True
    else:
        return False

# function to run the experiment
def experiment(days, p, s, endBet):
    # repeat for the number of days
    for i in range(days):
        # add to or subtract from s
        if dailyMovement(p):
            s += 1
        else:
            s -= 1
    # if s is greater than the end bet, you win
    if s > endBet:
        return True, s - endBet
    # otherwise, you lose
    else:
        return False, -1

s = 100
days = 30
endBet = 110
p = 0.5
N = 1000000
data = [experiment(days, p, s, endBet) for i in range(N)]

# wins keeps track of wins and losses
wins = []
# winsPayout keeps track of how much you win, but only if you win
winsPayout = []
for i in data:
    wins.append(i[0])
    if i[0]:
        winsPayout.append(i[1])

print(sum(wins)/N)
print(sum(winsPayout)/N)

0.02139
0.065932


No, this is a really terrible bet. The chances of winning are just over *2%* and the average payout is just 6 cents *when you do win*.

(b) If p = 0.6, what amount B would you be willing to pay to get in on the
game? Explain.

In [36]:
s = 100
days = 30
endBet = 110
p = 0.6
N = 1000000
data = [experiment(days, p, s, endBet) for i in range(N)]

wins = []
winsPayout = []
for i in data:
    wins.append(i[0])
    if i[0]:
        winsPayout.append(i[1])

print(sum(wins)/N)
print(sum(winsPayout)/N)

0.176443
0.679382


This is also a pretty bad bet. You lose over 4 of every 5 bets, and when you win, you win about 67 cents on average. Knowing you'd lose about 4 of every 5 bets, I would only pay a maximum of $0.67/5.5 \approx .1218$ cents to get into a game.

 3. Estimating a gradient: Given a function of two variables $f(x, y)$ that gives a
real-valued output, the gradient is a vector that represents a sort of derivative.
The important bit to know about the gradient is that it can be used to find the
direction of steepest decrease in the function output. If we are trying to minimize
$f$, this is great! So our goal is often to estimate the most negative gradient we
can.
To estimate the gradient of $f$ at point $(a, b)$ in the direction $(u, v)$ (where $u$ and
$v$ are small and not both zero), compute the ratio

$∇f (a, b) = \frac{f(a + u, b + v) − f (a, b)}{\sqrt{u^2 + v^2}}$

To find the direction of steepest descent, compute this many times for a variety
of $u$ and $v$ and keep track of the $(u, v)$ pair that gives you the most negative $∇f$ .
Here’s how to ”walk” to a minimum of $f$ using the method of steepest descent.
Begin with a start point, $(x_0, y_0)$. Find the direction of steepest descent of $f$,
giving you some pair $(u, v)$. Set $(x_1, y_1) = (x_0 + u, y_0 + v)$. Repeat, generating
new $x$ and $y$ values as you go. When they slow down you are nearing a minimum.
Implement the method of steepest descent and test it for the function $f(x, y) =
x^2 + 3y^2$, which has only one minimum value at $(x, y) = (0, 0)$.

In [41]:
# this calculates descent
def changeInF(x, y, u, v):
    return ((math.pow(x + u, 2) + 3 * math.pow(y + v, 2)) - (math.pow(x, 2) + 3 * math.pow(y, 2))) / math.sqrt(math.pow(u, 2) + math.pow(v, 2))

# This finds the point with the steepest descent out of a random 20 points
def findLowest(xy):
    x = xy[0]
    y = xy[1]
    tuples = []
    for i in range(20):
        u = .1 * (-1 + 2 * random.random())
        v = .1 * (-1 + 2 * random.random())
        tuples.append((u, v))
    current = (x, y)
    mostNegative = 0
    for i in tuples:
        if changeInF(x, y, i[0], i[1]) < mostNegative:
            current = (x + i[0], y + i[1])
            mostNegative = changeInF(x, y, i[0], i[1])
    return current

# this find the minimum given a random starting x and y
def findMinimum(N):
    startX = random.random() * 10 - 5
    startY = random.random() * 10 - 5
    xy = (startX, startY)
    for i in range(N):
        xy = findLowest(xy)
    return xy

findMinimum(1000000)

(-1.229629389695602e-05, 2.1272452370726445e-05)

(1.8611962299411625e-05, -1.9331451029785702e-05)

 5. The method of steepest descent can be used in many creative ways. For example,
let’s say you have two points in the plane, $(1, 10)$ and $(3, 0)$. We’d like to find
the slope $m$ and intercept $b$ of the line $y = mx + b$ through these two points, but
using the method of steepest descent rather than any algebra.
We will need a function $f(m, b)$ that reflects error. Let’s try something like

$f(m, b) = (m + b − 10)^2 + (3m + b − 0)^2$

Beginning with $m = 1$ and $b = 0$, use the method of steepest descent to estimate
$m$ and $b$. Check your work with a bit of algebra.

In [38]:
# This finds the error from the actual points given an m and b
def findError(m, b):
    return math.pow((m + b - 10), 2) + math.pow((3*m + b - 0), 2)

# This finds the m and b with the smallest error out of a series of 20 randomly generated m's and b's
def findSmallest(mb):
    m = mb[0]
    b = mb[1]
    tuples = []
    # randomly generate a series of u's and v's to find the next point to hop to
    for i in range(20):
        u = .1 * (-1 + 2 * random.random())
        v = .1 * (-1 + 2 * random.random())
        tuples.append((u, v))
    current = (m, b)
    err = abs(findError(m, b))
    for i in tuples:
        if abs(findError(m + i[0], b + i[1])) < err:
            current = (m + i[0], b + i[1])
            err = abs(findError(m + i[0], b + i[1]))
    return current

# This runs findSmallest N times to find the m and b with the smallest error
def findMinimum(N):
    mb = (1, 0)
    for i in range(N):
        mb = findSmallest(mb)
    return mb

findMinimum(10000000)

(-5.000009646849922, 15.00001973054796)

$m = (0 - 10) / (3 - 1) = -5$

$10 = -5 * 1 + b$ -> $b = 15$

5. Okay, arguably, that previous problem was not strictly necessary if we know the
magic algebra tricks. But what if the model we are working with is not a line?
For example, let’s say you have two points in the plane, (1, 10) and (3, 2). Let’s
also say that you want an exponential model $y = Ab^x$ that passes through these
points. Change f from the previous problem to reflect this new model and the
new point, and then use the method of steepest descent to estimate A and b. (You
should get A ≈ 22.361 and b ≈ 447.)

In [39]:
# This finds the error from the actual points given an A and b
def findError(A, b):
    return math.pow(A * math.pow(b, 1) - 10, 2) + math.pow(A * math.pow(b, 3) - 2, 2)

# This finds the A and b with the smallest error out of a series of 20 randomly generated A's and b's
def findSmallest(ab):
    A = ab[0]
    b = ab[1]
    tuples = []
    # randomly generate a series of u's and v's to find the next point to hop to
    for i in range(20):
        u = .1 * (-1 + 2 * random.random())
        v = .1 * (-1 + 2 * random.random())
        tuples.append((u, v))
    current = (A, b)
    err = abs(findError(A, b))
    for i in tuples:
        if abs(findError(A + i[0], b + i[1])) < err:
            current = (A + i[0], b + i[1])
            err = abs(findError(A + i[0], b + i[1]))
    return current

# This runs findSmallest N times to find the A and b with the smallest error
def findMinimum(N):
    Ab = (1, 0)
    for i in range(N):
        Ab = findSmallest(Ab)
    return Ab

findMinimum(10000000)

(22.36070173834083, 0.44721369316354814)